In [10]:
import numpy as np
import astropy.constants as aconst
import scipy.constants as const
from scipy.interpolate import interp1d
from pycbc.types import TimeSeries
from pycbc.filter import overlap_cplx
from pycbc.waveform import taper_timeseries
from pycbc.psd import aLIGOZeroDetHighPower
from pesummary.gw.conversions.mass import component_masses_from_mtotal_q
from simple_pe.waveforms import (
    orthonormalize_modes, calculate_mode_snr, two_ecc_harms_snr
)
from pesummary.gw.conversions.mass import m_total_from_m1_m2, q_from_m1_m2
from lalsimulation import SimAddMode

In [105]:
def generate_eccentric_waveform(
    m_total, q, e, chi1, chi2, f_gen, s_rate, phase=0, inclination=0,
    distance=1, f_ref_e=None, tlen=None, taper=True, to_fs=True,
    mode_array=[[2, 2]], align_merger=False, use_22_merger_time=True
):
    """
    Calculates eccentric teobresums waveform for the specified parameters.

    :param m_total: total mass
    :type m_total: float, required
    :param q: inverted mass ratio
    :type q: float, required
    :param e: eccentricity
    :type e: float, required
    :param chi1: aligned spin of primary object
    :type chi1: float, required
    :param chi2: aligned spin of secondary object
    :type chi1: float, required
    :param f_gen: waveform generation frequency
    :type f_gen: float, required
    :param s_rate: sample rate
    :type s_rate: float, required
    :param phase: coalescence phase
    :type phase: float, optional
    :param inclination: inclination of orbital plane
    :type inclination: float, optional
    :param distance: distance
    :type distance: float, optional
    :param f_ref_e: frequency at which eccentricity is defined (default f_gen)
    :type f_ref_e: float, optional
    :param tlen: length in seconds to resize waveforms to
    :type tlen: float, optional
    :param taper: flag to taper waveform
    :type taper: boolean, optional
    :param to_fs: flag to return waveform in frequency domain
    :type to_fs: boolean, optional
    :param mode_array: list of modes to include in waveform
    :type mode_array: list of lists of integer pairs, optional
    :param align_merger: whether to align merger to end of waveform
    :type align_merger: boolean, optional
    :param use_22_merger_time: whether to calculate merger time from 22 mode
                               if only single mode requested
    :type use_22_merger_time: boolean, optional

    :return h_plus: plus polarisations of waveform
    :rtype h_plus: TimeSeries or FrequencySeries object
    :return h_cross: cross polarisations of waveform
    :rtype h_cross: TimeSeries or FrequencySeries object
    """

    # Import teobresums
    try:
        import EOBRun_module
    except ImportError:
        print("Unable to import EOBRun_module, please check it is installed")
        return -1

    # Convert modes to teobresums convention
    if mode_array is None:
        mode_lm = [1]
        mode_array = [[2,2]]
    else:
        mode_lm = [int(x[0]*(x[0]-1)/2 + x[1]-2) for x in mode_array]
    if 1 not in mode_lm and len(mode_lm) == 1 and use_22_merger_time:
        mode_lm_gen = [1] + mode_lm
    else:
        mode_lm_gen = mode_lm
        use_22_merger_time=False

    # Convert eccentricity to correct frequency
    if f_ref_e is None:
        f_ref_e = f_gen
    else:
        e = shifted_e(f_gen, f_ref_e, e)

    # Convert Keplerian frequency to average frequency
    f_avg = f_gen*(1+e**2)/(1-e**2)**(3/2)

    # Generate waveform
    pars = {
        'M': m_total,
        'q': q,
        'chi1': chi1,
        'chi2': chi2,
        'LambdaAl2': 0.,
        'LambdaBl2': 0.,
        'ecc': e,
        'ecc_freq': 1,
        'domain': 0,
        'srate_interp': s_rate,
        'use_geometric_units': "no",
        'initial_frequency': f_avg,
        'interp_uniform_grid': "yes",
        'use_mode_lm': mode_lm_gen,
        'arg_out': "yes",
        'distance': distance,
        'coalescence_angle': phase,
        'inclination': inclination
    }
    t, hp, hc, hlm, _ = EOBRun_module.EOBRunPy(pars)

    # Calculate merger time
    if use_22_merger_time:
        tmrg = t[np.argmax(hlm['1'][0])]
        hp = hlm[str(mode_lm[0])][0]*np.cos(hlm[str(mode_lm[0])][1])
        hc = hlm[str(mode_lm[0])][0]*np.sin(hlm[str(mode_lm[0])][1])
    else:
         tmrg = t[np.argmax(np.abs(hp-1j*hc))]
    t = t - tmrg
    hp = TimeSeries(hp, t[1] - t[0], epoch=t[0])
    hc = TimeSeries(hc, t[1] - t[0], epoch=t[0]) 

    # Resize waveforms, taper, and return in requested domain
    if tlen is not None:
        hp.resize(tlen*s_rate)
        hc.resize(tlen*s_rate)
    if taper:
        hp = taper_timeseries(hp, tapermethod='start')
        hc = taper_timeseries(hc, tapermethod='start')
    if align_merger:
        hp = hp.cyclic_time_shift(hp.sample_times[-1])
        hc = hc.cyclic_time_shift(hc.sample_times[-1])
    if to_fs:
        hp = hp.to_frequencyseries()
        hc = hc.to_frequencyseries()
    return hp, hc

In [113]:
hp, hc = generate_eccentric_waveform(
    60, 2, 0, 0, 0, 10, 4096, phase=np.pi/2, inclination=np.pi/2,
    distance=87, f_ref_e=None, tlen=None, taper=True, to_fs=True,
    mode_array=[[2, 2]], align_merger=True
)
hp[100]

(-1.596143253350087e-22+2.0927322537797461e-22j)

In [107]:
hp, hc = generate_eccentric_waveform(
    60, 2, 0, 0, 0, 10, 4096, phase=np.pi/2, inclination=np.pi/2,
    distance=87, f_ref_e=None, tlen=None, taper=True, to_fs=True,
    mode_array=[[3, 3]], align_merger=True
)
print(hp.to_timeseries().sample_times[0])

-6.6979980470000005
